In [1]:
import pandas as pd

In [13]:
df = pd.read_csv('https://raw.githubusercontent.com/Alois-Software/data-modeling/word-embedding/social_media_parsed.csv', header=0)

In [15]:
df

,text
0,🌼 Alzheimers Dementia and Support ‘ADS’ is one...
1,"Hello Everyone,\nIt is that time of year again..."
2,Slough Borough Council is seeking the views of...
3,Not heard from us in a while?\n\nWe are not ig...
4,Less than 3months to go...\n\nGet yourself rea...
...,...
520,I've been doing a lot of thinking about what i...
521,"I needed this today, on Father’s Day."
522,Mom just told me she don't feel good. She tell...
523,Wishing you a rainbow of happiness and good ch...


In [16]:
df['text']

0      🌼 Alzheimers Dementia and Support ‘ADS’ is one...
1      Hello Everyone,\nIt is that time of year again...
2      Slough Borough Council is seeking the views of...
3      Not heard from us in a while?\n\nWe are not ig...
4      Less than 3months to go...\n\nGet yourself rea...
                             ...                        
520    I've been doing a lot of thinking about what i...
521                I needed this today, on Father’s Day.
522    Mom just told me she don't feel good. She tell...
523    Wishing you a rainbow of happiness and good ch...
524    The Red Arrows flying over Maidenhead after th...
Name: text, Length: 525, dtype: object